# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [2]:
#Your code here; create the connection and cursor
conn = sqlite3.Connection('data.sqlite')
c = conn.cursor()

## Select the Average Number of Orders on a Product by Product Basis

In [3]:
#Your code here; use a subquery. No join will be necessary.

c.execute("""select AVG(numOrders)
                    from (select productCode,
                                 count(distinct orderNumber) as numOrders
                                 from products
                                 join orderdetails
                                 using(productCode)
                                 group by 1
                         );
          """
         )

#c.execute("""select AVG(sum(quantityOrdered) / count(productCode) as avg_orders,
#                    productCode 
#                    from orderdetails
#                    group by productCode;
#          """
#         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,AVG(numOrders)
0,27.486239


## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [4]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""select customer, first, last
                    from (select customerNumber as customer, 
                                 contactLastName as last,
                                 contactFirstName as first
                                 from customers
                                 join orders
                                 using(customerNumber)
                                 where orderDate = '2003-01-31');""").fetchall() 

[('141', 'Diego ', 'Freyre')]

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [5]:
#Your code here
c.execute("""select productName, count(distinct orderNumber) as numberOrders,
                                 sum(quantityOrdered) as totalUnitsSold
                                 from products
                                 join orderdetails
                                 using(productCode)
                                 group by 1
                                 order by TotalUnitsSold desc;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()


,productName,numberOrders,totalUnitsSold
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [68]:
#Your code here

c.execute("""SELECT N.productName,
                    sum(O.quantityOrdered) as Units_Ordered
                    FROM orderdetails O
                    JOIN (select productCode, productName from products) N
                    USING(productCode)
                    GROUP BY 1
                    ORDER BY
                    productCode
                    ;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()


,productName,Units_Ordered
0,1969 Harley Davidson Ultimate Chopper,1057
1,1952 Alpine Renault 1300,961
2,1996 Moto Guzzi 1100i,999
3,2003 Harley-Davidson Eagle Drag Bike,985
4,1972 Alfa Romeo GTA,1030


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [154]:
#Your code here
c.execute("""SELECT distinct E.employeeNumber as EmpNum,
                    E.officeCode as OfficeCode,
                    S.city,
                    E.firstName as EmpFirstName,
                    E.lastName as EmpLastName
                    FROM orderdetails D
                    LEFT JOIN products P ON (P.productCode = D.productCode)
                    LEFT JOIN orders O ON (O.orderNumber = D.orderNumber)
                    LEFT JOIN customers C ON (C.customerNumber = O.customerNumber)
                    LEFT JOIN employees E ON (E.employeeNumber = C.salesRepEmployeeNumber)
                    LEFT JOIN offices S ON (S.officeCode = E.officeCode)
                    WHERE (select productCode
                                  from products
                                  join orderdetails
                                  using (productCode)
                                  join orders
                                  using (orderNumber)
                                  group by 1
                                  having count(distinct customerNumber) <= 20)
                    ;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()


ValueError: Length mismatch: Expected axis has 0 elements, new values have 5 elements

## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [ ]:
#Your code here

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!